In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from api_key import gkey
from scipy.stats import linregress


In [2]:
crime = "Resources/UCR_Part_1_Crime_for_Multiple_St._Louis_County_Police_Departments%2C_2019_Year_to_Date.csv"
crime_df = pd.read_csv(crime)
#crime_df

In [3]:
crime_df.columns


Index(['X', 'Y', 'UCRCOUNT', 'COMPLAINTYEAR', 'COMPLAINTNUM', 'UCR_OFFENSE',
       'UCR_CRIME_CODE', 'TYPE', 'MONTH', 'YEAR', 'DT_CALLREC', 'D_OCCURRED',
       'DOW_OCCURRED', 'ZONE', 'ADDRESS', 'PRECINCT', 'PREMISE',
       'REPORTING_JURIS', 'FOR_JURIS', 'X.1', 'Y.1', 'OBJECTID'],
      dtype='object')

In [4]:
reduced_crime_df = crime_df.loc[:, ['X', 'Y', 'COMPLAINTYEAR', 'UCR_OFFENSE', 'TYPE', 'YEAR',
                                    'ADDRESS']]
reduced_crime_df

,X,Y,COMPLAINTYEAR,UCR_OFFENSE,TYPE,YEAR,ADDRESS
0,-90.303477,38.680753,18,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,1439 KINGSLAND AVE
1,-90.271928,38.720354,18,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,7357 JENWOOD AVE
2,-90.268498,38.703995,19,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,5200 HELEN AVE
3,-90.256474,38.725830,18,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,8815 CLIFTON AVE
4,-90.215647,38.773309,18,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,11137 RIAZA SQ
...,...,...,...,...,...,...,...
17594,NaN,NaN,15,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,577 GODDARD AVE
17595,-90.546271,38.606776,19,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,328 NORWICH CT
17596,-90.295168,38.543718,19,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,4190 HOFFMEISTER AVE
17597,-90.448450,38.539104,19,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,1710 FENPARK DR


In [5]:
year_df = reduced_crime_df.loc[reduced_crime_df['COMPLAINTYEAR'] >= 19]
year_df

,X,Y,COMPLAINTYEAR,UCR_OFFENSE,TYPE,YEAR,ADDRESS
2,-90.268498,38.703995,19,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,5200 HELEN AVE
5,-90.197312,38.753107,19,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,CAMERON RD & ROSS CIR
6,-90.182395,38.777746,19,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,610 PRIGGE RD
7,NaN,NaN,19,"RAPE, BY FORCE (AFTER 12-31-2012)",PERSON,2019,NaN
8,NaN,NaN,19,"RAPE, BY FORCE (AFTER 12-31-2012)",PERSON,2019,NaN
...,...,...,...,...,...,...,...
17593,-90.245846,38.745910,19,"MOTOR VEHICLE THEFT, TRUCK/BUS",PROPERTY,2019,9810 DUKE DR
17595,-90.546271,38.606776,19,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,328 NORWICH CT
17596,-90.295168,38.543718,19,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,4190 HOFFMEISTER AVE
17597,-90.448450,38.539104,19,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,1710 FENPARK DR


In [6]:
ryear_df = year_df.loc[:, ['X', 'Y', 'UCR_OFFENSE', 'TYPE', 'YEAR',
                                    'ADDRESS']]
ryear_df.dropna(inplace = True) 

ryear_df = ryear_df.reset_index(drop=True)
ryear_df

,X,Y,UCR_OFFENSE,TYPE,YEAR,ADDRESS
0,-90.268498,38.703995,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,5200 HELEN AVE
1,-90.197312,38.753107,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,CAMERON RD & ROSS CIR
2,-90.182395,38.777746,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,610 PRIGGE RD
3,-90.470593,38.540136,"ROBBERY, FIREARM",PERSON,2019,1860 BOWLES AVE
4,-90.449337,38.614038,"ROBBERY, FIREARM",PERSON,2019,1909 WINDMOOR PL
...,...,...,...,...,...,...
16453,-90.245846,38.745910,"MOTOR VEHICLE THEFT, TRUCK/BUS",PROPERTY,2019,9810 DUKE DR
16454,-90.546271,38.606776,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,328 NORWICH CT
16455,-90.295168,38.543718,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,4190 HOFFMEISTER AVE
16456,-90.448450,38.539104,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,1710 FENPARK DR


In [7]:
ryear_df['Zipcode'] = ""
ryear_df.head()

,X,Y,UCR_OFFENSE,TYPE,YEAR,ADDRESS,Zipcode
0,-90.268498,38.703995,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,5200 HELEN AVE,
1,-90.197312,38.753107,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,CAMERON RD & ROSS CIR,
2,-90.182395,38.777746,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,610 PRIGGE RD,
3,-90.470593,38.540136,"ROBBERY, FIREARM",PERSON,2019,1860 BOWLES AVE,
4,-90.449337,38.614038,"ROBBERY, FIREARM",PERSON,2019,1909 WINDMOOR PL,


In [8]:
# # TEST DATA
# ryear_df_test = ryear_df.head()
# locations = ryear_df_test[["X", "Y"]]

# #locations = ryear_df[["X", "Y"]]

# #geo_data = requests.get(target_url).json()
# type(locations)
# params = {"key": gkey}
# #for i in locations:
#    # print(i)
# master_zips= []

In [221]:
# # TEST DATA
# #ryear_df_test = ryear_df.head(5).copy()
# #locations = ryear_df_test[["X", "Y", 'Zipcode']]

# locations = ryear_df[["X", "Y", 'Zipcode']]

# #geo_data = requests.get(target_url).json()
# type(locations)
# params = {"key": gkey}
# #for i in locations:
#    # print(i)
# master_zips= []
# try:
#     for index, data in locations.iterrows():
#         if  data['Zipcode'] == '':

#             url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={data["Y"]},{data["X"]}&key={params["key"]}'
#             #print(url)
#             response = requests.get(url).json()
#             zip_list = []
#             results = response['results']


#             for result in results:
#             #print(result)
#                 if result.get('address_components'):
#                     for component in result.get('address_components'):
#                         #print(component)
#                         ctype = (component.get("types"))[0]
#                         if ctype == 'postal_code':
#                             current_zip = component.get('short_name')
#                             zip_list.append(current_zip)

#                 zip_list = list(set(zip_list))

#           #  master_zips.append(zip_list)
#             ryear_df.loc[index, 'Zipcode'] = zip_list[0]
# except ValueError:
#     print('There is something wrong with this zipcode!')

In [225]:
cryear_df = ryear_df.copy()

In [224]:
ryear_df.head(17000)


,X,Y,UCR_OFFENSE,TYPE,YEAR,ADDRESS,Zipcode
0,-90.268498,38.703995,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,5200 HELEN AVE,63136
1,-90.197312,38.753107,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,CAMERON RD & ROSS CIR,63137
2,-90.182395,38.777746,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,610 PRIGGE RD,63138
3,-90.470593,38.540136,"ROBBERY, FIREARM",PERSON,2019,1860 BOWLES AVE,63026
4,-90.449337,38.614038,"ROBBERY, FIREARM",PERSON,2019,1909 WINDMOOR PL,63131
...,...,...,...,...,...,...,...
16453,-90.245846,38.745910,"MOTOR VEHICLE THEFT, TRUCK/BUS",PROPERTY,2019,9810 DUKE DR,63136
16454,-90.546271,38.606776,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,328 NORWICH CT,63011
16455,-90.295168,38.543718,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,4190 HOFFMEISTER AVE,63125
16456,-90.448450,38.539104,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,1710 FENPARK DR,63026


In [227]:
nryear_df = ryear_df.loc[:, ['UCR_OFFENSE', 'TYPE', 'YEAR',
                                    'ADDRESS', 'Zipcode']]
nryear_df

,UCR_OFFENSE,TYPE,YEAR,ADDRESS,Zipcode
0,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,5200 HELEN AVE,63136
1,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,CAMERON RD & ROSS CIR,63137
2,"HOMICIDE, MURDER AND NONNEGLIGENT MANSLAUGHTER",PERSON,2019,610 PRIGGE RD,63138
3,"ROBBERY, FIREARM",PERSON,2019,1860 BOWLES AVE,63026
4,"ROBBERY, FIREARM",PERSON,2019,1909 WINDMOOR PL,63131
...,...,...,...,...,...
16453,"MOTOR VEHICLE THEFT, TRUCK/BUS",PROPERTY,2019,9810 DUKE DR,63136
16454,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,328 NORWICH CT,63011
16455,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,4190 HOFFMEISTER AVE,63125
16456,"MOTOR VEHICLE THEFT, OTHER VEHICLE",PROPERTY,2019,1710 FENPARK DR,63026
